<a href="https://colab.research.google.com/github/talhadary/adaboostTitanic/blob/master/adaBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from google.colab import files
uploaded = files.upload()

Saving Titanic-Dataset.csv to Titanic-Dataset (5).csv


In [60]:
import pandas as pd

# Load the CSV
df = pd.read_csv("Titanic-Dataset.csv")
print(df.head())
print(df.isnull().sum())


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
Pa

In [61]:
# dropping useless columns
df = df.drop(columns=["Cabin", "Ticket", "Name", "PassengerId"])

# filling empty data. age with median, embarked with S
df.fillna({"Age": df["Age"].median()}, inplace = True)
df.fillna({"Embarked": df["Embarked"].mode()[0]}, inplace = True)

# convert categories to integers
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})
df["Embarked"] = df["Embarked"].map({"S": 0, "C": 1, "Q": 2})

In [62]:
# feature target split
X = df.drop(columns=["Survived"])
y = df["Survived"]

# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=66)

In [63]:
# making dummy models
from sklearn.dummy import DummyClassifier

dummy_const = DummyClassifier(strategy="constant", constant=0)
dummy_const.fit(X_train, y_train)
y_pred = dummy_const.predict(X_test)
print("Dummy_conts: ", accuracy_score(y_test, y_pred))

dummy_uni = DummyClassifier(strategy="uniform")
dummy_uni.fit(X_train, y_train)
y_pred = dummy_uni.predict(X_test)
print("Dummy_uni: ", accuracy_score(y_test, y_pred))

Dummy_conts:  0.6457399103139013
Dummy_uni:  0.47533632286995514


In [64]:
# fitting initial model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

base = DecisionTreeClassifier(max_depth=1)
model = AdaBoostClassifier(n_estimators=10, learning_rate=1.0, random_state=66)
model.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=10, random_state=66)

In [65]:
# evalutate initial model
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7847533632286996
              precision    recall  f1-score   support

           0       0.83      0.83      0.83       144
           1       0.70      0.70      0.70        79

    accuracy                           0.78       223
   macro avg       0.76      0.76      0.76       223
weighted avg       0.78      0.78      0.78       223



In [66]:
# cross validation to fine tune parameters
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [50, 100, 200],
    "learning_rate": [0.001, 0.01, 0.1]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)
print("Best cross-val accuracy:", grid_search.best_score_)

Best parameters: {'learning_rate': 0.1, 'n_estimators': 100}
Best cross-val accuracy: 0.7918527662439682
